In [ ]:
# # THIS MAKES A LIST OF LAT LONG
# import rasterio
# from rasterio.features import geometry_mask
# from shapely.geometry import Point, box
# from geopandas import GeoDataFrame
# import numpy as np

# # Load the GeoTIFF file
# file_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\Input\pcl_west_wgs_CO_3.tif"
# with rasterio.open(file_path) as src:
#     # Extract the bounds of the raster
#     bounds = src.bounds
#     crs = src.crs

# # Create a box from the bounds
# rect = box(bounds.left, bounds.bottom, bounds.right, bounds.top)

# # Apply a negative buffer to contract the boundary of the rectangle
# buffered_rect = rect.buffer(-0.00001)  # Change 1000 to your desired buffer distance in units of the CRS

# # Check if the buffered rectangle is empty (can happen with large negative buffers)
# if buffered_rect.is_empty:
#     raise ValueError("Buffer size is too large")

# # Function to generate random points within a geometry
# def generate_random_points(geometry, num_points):
#     points = []
#     min_x, min_y, max_x, max_y = geometry.bounds
#     while len(points) < num_points:
#         random_point = Point(np.random.uniform(min_x, max_x), np.random.uniform(min_y, max_y))
#         if random_point.within(geometry):
#             points.append(random_point)
#     return points

# # Generate random points within the buffered rectangle
# random_points = generate_random_points(buffered_rect, 500)

# # Convert these points to a GeoDataFrame
# gdf_points = GeoDataFrame(geometry=random_points, crs=crs)

# # Convert geometries from the CRS to WGS84 for latitude and longitude
# gdf_points_wgs84 = gdf_points.to_crs(epsg=4326)

# # Extract the latitude and longitude
# lat_long = gdf_points_wgs84.geometry.apply(lambda geom: (geom.y, geom.x)).tolist()

# # # Print the latitude and longitude coordinates
# # for lat, lon in lat_long:
# #     print(f"Latitude: {lat}, Longitude: {lon}")
# print("done")

In [ ]:
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show
from shapely.geometry import box, Point
from geopandas import GeoDataFrame
import numpy as np

# Load the GeoTIFF file
file_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\Input\pcl_west_wgs_CO_3.tif"
with rasterio.open(file_path) as src:
    bounds = src.bounds
    crs = src.crs
    img = src.read(1)  # Read the first band

# Create a box from the bounds
rect = box(bounds.left, bounds.bottom, bounds.right, bounds.top)

# Apply a negative buffer to contract the boundary of the rectangle
buffered_rect = rect.buffer(-0.15)  # Adjust buffer size as needed

# Generate random points within the buffered rectangle
def generate_random_points(geometry, num_points):
    points = []
    min_x, min_y, max_x, max_y = geometry.bounds
    while len(points) < num_points:
        random_point = Point(np.random.uniform(min_x, max_x), np.random.uniform(min_y, max_y))
        if random_point.within(geometry):
            points.append(random_point)
    return points

random_points = generate_random_points(buffered_rect, 20000)

# Convert these points to a GeoDataFrame
gdf_points = GeoDataFrame(geometry=random_points, crs=crs)

# Convert points GeoDataFrame to the same CRS as the raster for accurate overlay
gdf_points = gdf_points.to_crs(crs)

# Convert geometries from the CRS to WGS84 for latitude and longitude
gdf_points_wgs84 = gdf_points.to_crs(epsg=4326)

# Extract the latitude and longitude
lat_long = gdf_points_wgs84.geometry.apply(lambda geom: (geom.y, geom.x)).tolist()

# Plotting the raster and the points
fig, ax = plt.subplots(figsize=(10, 10))
show(img, ax=ax, transform=src.transform, cmap='gray')  # Show the raster
gdf_points.plot(ax=ax, color='red', markersize=10)  # Plot the points
plt.title('Random Points on Raster')
plt.show()


In [ ]:
import rasterio
from rasterio.windows import Window

# Paths to the raster file
resampled_PCL_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\Input\pcl_west_wgs_CO_3.tif"
resampled_lidar_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\Input\Resampled_LIDAR_GEDI_CO_11.tif"
resampled_dem_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\Input\Resampled_DEM_CO.tif"
resampled_rivers_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\Input\Resampled_RIVERS_CO.tif"
resampled_roads_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\Input\Resampled_ROADS_CO.tif"

# Size of the chip around the point (in pixels)
chip_size = 64  

for i in range(len(random_points)):
    try:
        # Define the coordinates of the point (replace these with your actual coordinates)
        x_coord = lat_long[i][1] 
        y_coord = lat_long[i][0]
        
        # Open the resampled LIDAR raster
        with rasterio.open(resampled_PCL_path) as pcl:
            # Convert the geographic coordinates to pixel coordinates
            col, row = pcl.index(x_coord, y_coord)
            
            # Calculate the window
            window = Window(col - chip_size, row - chip_size, 2 * chip_size, 2 * chip_size)
            
            # Read the data within the window
            chip_data = pcl.read(window=window)
            
            # Prepare metadata for the chip raster
            out_meta = pcl.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": chip_data.shape[1],
                "width": chip_data.shape[2],
                "transform": pcl.window_transform(window)
            })
        
            # Save the chip data to a new file
            chip_output_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\labels4\\PCL_Chip_{i}.tif"
            #with rasterio.open(chip_output_path, "w", **out_meta) as dest:
            #    dest.write(chip_data)
            #print(chip_data.shape)
            try:
                if chip_data.shape == (1,2 * chip_size,2 * chip_size) and not np.allclose(chip_data, 0):
                    with rasterio.open(chip_output_path, "w", **out_meta) as dest:
                        dest.write(chip_data)
                else:
                    print(f"Skipping chip {i} because its shape is not 200x200.")
            except Exception as e:
                print(f"Error occurred while saving chip {i}: {e}")
        
        #print("Image chip created and saved to:", chip_output_path)
        
        with rasterio.open(resampled_lidar_path) as lidar:
            # Convert the geographic coordinates to pixel coordinates
            col, row = lidar.index(x_coord, y_coord)
            
            # Calculate the window
            window = Window(col - chip_size, row - chip_size, 2 * chip_size, 2 * chip_size)
            
            # Read the data within the window
            chip_data = lidar.read(window=window)
            
            # Prepare metadata for the chip raster
            out_meta = lidar.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": chip_data.shape[1],
                "width": chip_data.shape[2],
                "transform": lidar.window_transform(window)
            })
        
            # Save the chip data to a new file
            chip_output_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inputfeatures4\\hag\\LIDAR_Chip_{i}.tif"
            #with rasterio.open(chip_output_path, "w", **out_meta) as dest:
            #    dest.write(chip_data)
            try:
                if chip_data.shape == (1,2 * chip_size,2 * chip_size) and not np.isnan(chip_data).all():
                    with rasterio.open(chip_output_path, "w", **out_meta) as dest:
                        dest.write(chip_data)
                else:
                    print(f"Skipping chip {i} because its shape is not 200x200.")
            except Exception as e:
                print(f"Error occurred while saving chip {i}: {e}")

        with rasterio.open(resampled_dem_path) as dem:
            # Convert the geographic coordinates to pixel coordinates
            col, row = dem.index(x_coord, y_coord)
            
            # Calculate the window
            window = Window(col - chip_size, row - chip_size, 2 * chip_size, 2 * chip_size)
            
            # Read the data within the window
            chip_data = dem.read(window=window)
            
            # Prepare metadata for the chip raster
            out_meta = dem.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": chip_data.shape[1],
                "width": chip_data.shape[2],
                "transform": dem.window_transform(window)
            })
        
            # Save the chip data to a new file
            chip_output_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inputfeatures4\\dem\\DEM_Chip_{i}.tif"
            #with rasterio.open(chip_output_path, "w", **out_meta) as dest:
            #    dest.write(chip_data)
            try:
                if chip_data.shape == (1,2 * chip_size,2 * chip_size) and not np.isnan(chip_data).all():
                    with rasterio.open(chip_output_path, "w", **out_meta) as dest:
                        dest.write(chip_data)
                else:
                    print(f"Skipping chip {i} because its shape is not 200x200.")
            except Exception as e:
                print(f"Error occurred while saving chip {i}: {e}")

        with rasterio.open(resampled_rivers_path) as rivers:
            # Convert the geographic coordinates to pixel coordinates
            col, row = rivers.index(x_coord, y_coord)
            
            # Calculate the window
            window = Window(col - chip_size, row - chip_size, 2 * chip_size, 2 * chip_size)
            
            # Read the data within the window
            chip_data = rivers.read(window=window)
            
            # Prepare metadata for the chip raster
            out_meta = rivers.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": chip_data.shape[1],
                "width": chip_data.shape[2],
                "transform": rivers.window_transform(window)
            })
        
            # Save the chip data to a new file
            chip_output_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inputfeatures4\\rivers\\River_Chip_{i}.tif"

            try:
                if chip_data.shape == (1,2 * chip_size,2 * chip_size) and not np.isnan(chip_data).all():
                    with rasterio.open(chip_output_path, "w", **out_meta) as dest:
                        dest.write(chip_data)
                else:
                    print(f"Skipping chip {i} because its shape is not 200x200.")
            except Exception as e:
                print(f"Error occurred while saving chip {i}: {e}")

        with rasterio.open(resampled_rivers_path) as roads:
            # Convert the geographic coordinates to pixel coordinates
            col, row = roads.index(x_coord, y_coord)
            
            # Calculate the window
            window = Window(col - chip_size, row - chip_size, 2 * chip_size, 2 * chip_size)
            
            # Read the data within the window
            chip_data = roads.read(window=window)
            
            # Prepare metadata for the chip raster
            out_meta = roads.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": chip_data.shape[1],
                "width": chip_data.shape[2],
                "transform": roads.window_transform(window)
            })
        
            # Save the chip data to a new file
            chip_output_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inputfeatures4\\roads\\Roads_Chip_{i}.tif"

            try:
                if chip_data.shape == (1,2 * chip_size,2 * chip_size) and not np.isnan(chip_data).all():
                    with rasterio.open(chip_output_path, "w", **out_meta) as dest:
                        dest.write(chip_data)
                else:
                    print(f"Skipping chip {i} because its shape is not 200x200.")
            except Exception as e:
                print(f"Error occurred while saving chip {i}: {e}")

        
        print("Image chip created and saved to:", chip_output_path)
    except:
        continue

In [ ]:
# import os
# import rasterio
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# # Paths to datasets
# featurepath1 = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\inputfeatures4\hag"
# featurepath2 = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\inputfeatures4\dem"
# featurepath2 = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\inputfeatures4\roads"
# featurepath2 = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\inputfeatures4\rivers"

# labelspath = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\labels4"

# # Function to load GeoTIFF images as numpy arrays
# def load_geotiff(path):
#     with rasterio.open(path) as src:
#         return src.read(1)
# print("1")
# # Load datasets
# hag_images = [load_geotiff(os.path.join(featurepath1, f)) for f in os.listdir(featurepath1) if f.endswith('.tif')]
# dem_images = [load_geotiff(os.path.join(featurepath2, f)) for f in os.listdir(featurepath2) if f.endswith('.tif')]
# label_images = [load_geotiff(os.path.join(labelspath, f)) for f in os.listdir(labelspath) if f.endswith('.tif')]
# print("2")

# # Convert lists to numpy arrays and stack features along the last dimension
# hag_images = np.array(hag_images).astype('float32')
# dem_images = np.array(dem_images).astype('float32')
# feature_images = np.stack((hag_images, dem_images), axis=-1)  # Stack along the channel axis

# label_images = np.array(label_images).astype('float32')
# print("3")

# # Normalize feature images to range [0, 1] and label images to a specific range
# feature_max = np.max(feature_images, axis=(1, 2, 3), keepdims=True)
# feature_images /= feature_max
# label_images /= 100  # Normalize labels if they range from 0 to 100

# # Reshape labels for CNN input
# label_images = np.expand_dims(label_images, axis=-1)
# print("4")

# # Define the CNN model
# model = Sequential([
#     Conv2D(16, (3, 3), activation='relu', input_shape=(128, 128, 4)),  # Adjust input shape for two channels
#     MaxPooling2D((2, 2)),
#     Dropout(0.25),
#     Conv2D(32, (3, 3), activation='relu'),
#     MaxPooling2D((2, 2)),
#     Dropout(0.25),
#     Conv2D(64, (3, 3), activation='relu'),
#     Flatten(),
#     Dropout(0.5),
#     Dense(128 * 128, activation='sigmoid'),
#     tf.keras.layers.Reshape((128, 128, 1))
# ])

# # Compile and train the model
# model.compile(optimizer='adam', loss='mse')
# model.fit(feature_images, label_images, batch_size=64, epochs=1, validation_split=0.1)


In [ ]:
import os
import rasterio
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Paths to datasets
featurepath1 = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\inputfeatures4\hag"
featurepath2 = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\inputfeatures4\dem"
featurepath3 = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\inputfeatures4\roads"
featurepath4 = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\inputfeatures4\rivers"




labelspath = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\labels4"

# Function to load GeoTIFF images as numpy arrays
def load_geotiff(path):
    with rasterio.open(path) as src:
        return src.read(1)

# Load datasets
hag_images = [load_geotiff(os.path.join(featurepath1, f)) for f in os.listdir(featurepath1) if f.endswith('.tif')]
dem_images = [load_geotiff(os.path.join(featurepath2, f)) for f in os.listdir(featurepath2) if f.endswith('.tif')]
roads_images = [load_geotiff(os.path.join(featurepath3, f)) for f in os.listdir(featurepath3) if f.endswith('.tif')]
rivers_images = [load_geotiff(os.path.join(featurepath4, f)) for f in os.listdir(featurepath4) if f.endswith('.tif')]

label_images = [load_geotiff(os.path.join(labelspath, f)) for f in os.listdir(labelspath) if f.endswith('.tif')]

# Convert lists to numpy arrays
hag_images = np.array(hag_images).astype('float32')
dem_images = np.array(dem_images).astype('float32')
roads_images = np.array(roads_images).astype('float32')
rivers_images = np.array(rivers_images).astype('float32')

label_images = np.array(label_images).astype('float32')

# Normalize images independently
hag_max = hag_images.max()
dem_max = dem_images.max()
roads_max = roads_images.max()
rivers_max = rivers_images.max()


hag_images /= hag_max
dem_images /= dem_max
roads_images /= roads_max
rivers_images /= rivers_max


# Stack features along the last dimension
feature_images = np.stack((hag_images, dem_images, roads_images, rivers_images), axis=-1)

# Normalize labels if they range from 0 to 100
label_images /= 100

# Reshape labels for CNN input
label_images = np.expand_dims(label_images, axis=-1)

# Define the CNN model
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(128, 128, 4)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dropout(0.5),
    Dense(128 * 128, activation='sigmoid'),
    tf.keras.layers.Reshape((128, 128, 1))
])

lr = 0.01
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Compile and train the model
model.compile(optimizer=optimizer, loss='mse')
model.fit(feature_images, label_images, batch_size=128, epochs=10, validation_split=0.3)


In [ ]:
#model.save('C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\SavedModels\\model_4_30_LR01.h5')  # Saves the model in HDF5 format

from tensorflow.keras.models import load_model

# Path to the model
model_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\SavedModels\\model_4_30_LR01.h5'

# Load the model
model = load_model(model_path)


In [ ]:
# this ensures all tiles are widthtile by heighttile
import os
from itertools import product
import rasterio as rio
from rasterio import windows




in_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\Input\\'
input_filename1 = 'Resampled_LIDAR_GEDI_CO_11.tif'
input_filename2 = 'Resampled_DEM_CO.tif'
input_filename3 = 'Resampled_ROADS_CO.tif'
input_filename4 = 'Resampled_RIVERS_CO.tif'



out_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\hag\\'
output_filename = 'tile_{}-{}.tif'
widthtile = 128
heighttile = 128


def get_tiles(ds, width=widthtile, height=heighttile):
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, width), range(0, nrows, height))
    #offsets = product(range(0, nols, 1000), range(0, nrows, 1000))

    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in offsets:
        window = windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform




with rio.open(os.path.join(in_path, input_filename1)) as inds:
    tile_width, tile_height = widthtile, heighttile
    nodata = inds.nodata  # Get the NoData value from the dataset
    meta = inds.meta.copy()

    for window, transform in get_tiles(inds):
        if window.width == tile_width and window.height == tile_height:  # Check if the tile dimensions are as expected
            data = inds.read(window=window)
            if nodata is not None:
                # Modified check for NoData to include tolerance for floating-point rasters
                valid_data_mask = (data != nodata)
            else:
                # If NoData value is not set, consider all data as valid
                valid_data_mask = (data == data)

            if valid_data_mask.any():  # Check if there's any valid data within the tile
                meta['transform'] = transform
                meta['width'], meta['height'] = window.width, window.height
                outpath = os.path.join(out_path, output_filename.format(int(window.col_off), int(window.row_off)))
                with rio.open(outpath, 'w', **meta) as outds:
                    outds.write(data)



out_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\dem\\'
#output_filename = 'tile_{}-{}.tif'
#widthtile = 128
#heighttile = 128

with rio.open(os.path.join(in_path, input_filename2)) as inds:
    tile_width, tile_height = widthtile, heighttile
    nodata = inds.nodata  # Get the NoData value from the dataset
    meta = inds.meta.copy()

    for window, transform in get_tiles(inds):
        if window.width == tile_width and window.height == tile_height:  # Check if the tile dimensions are as expected
            data = inds.read(window=window)
            if nodata is not None:
                # Modified check for NoData to include tolerance for floating-point rasters
                valid_data_mask = (data != nodata)
            else:
                # If NoData value is not set, consider all data as valid
                valid_data_mask = (data == data)

            if valid_data_mask.any():  # Check if there's any valid data within the tile
                meta['transform'] = transform
                meta['width'], meta['height'] = window.width, window.height
                outpath = os.path.join(out_path, output_filename.format(int(window.col_off), int(window.row_off)))
                with rio.open(outpath, 'w', **meta) as outds:
                    outds.write(data)

out_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\roads\\'
#output_filename = 'tile_{}-{}.tif'
#widthtile = 128
#heighttile = 128

with rio.open(os.path.join(in_path, input_filename3)) as inds:
    tile_width, tile_height = widthtile, heighttile
    nodata = inds.nodata  # Get the NoData value from the dataset
    meta = inds.meta.copy()

    for window, transform in get_tiles(inds):
        if window.width == tile_width and window.height == tile_height:  # Check if the tile dimensions are as expected
            data = inds.read(window=window)
            if nodata is not None:
                # Modified check for NoData to include tolerance for floating-point rasters
                valid_data_mask = (data != nodata)
            else:
                # If NoData value is not set, consider all data as valid
                valid_data_mask = (data == data)

            if valid_data_mask.any():  # Check if there's any valid data within the tile
                meta['transform'] = transform
                meta['width'], meta['height'] = window.width, window.height
                outpath = os.path.join(out_path, output_filename.format(int(window.col_off), int(window.row_off)))
                with rio.open(outpath, 'w', **meta) as outds:
                    outds.write(data)

out_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\rivers\\'
#output_filename = 'tile_{}-{}.tif'
#widthtile = 128
#heighttile = 128

with rio.open(os.path.join(in_path, input_filename4)) as inds:
    tile_width, tile_height = widthtile, heighttile
    nodata = inds.nodata  # Get the NoData value from the dataset
    meta = inds.meta.copy()

    for window, transform in get_tiles(inds):
        if window.width == tile_width and window.height == tile_height:  # Check if the tile dimensions are as expected
            data = inds.read(window=window)
            if nodata is not None:
                # Modified check for NoData to include tolerance for floating-point rasters
                valid_data_mask = (data != nodata)
            else:
                # If NoData value is not set, consider all data as valid
                valid_data_mask = (data == data)

            if valid_data_mask.any():  # Check if there's any valid data within the tile
                meta['transform'] = transform
                meta['width'], meta['height'] = window.width, window.height
                outpath = os.path.join(out_path, output_filename.format(int(window.col_off), int(window.row_off)))
                with rio.open(outpath, 'w', **meta) as outds:
                    outds.write(data)

print("Done")


In [ ]:
# import os
# import rasterio
# import numpy as np
# import tensorflow as tf

# tilename = '0-0'
# input_image_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\HAGTiles3\\tile_{tilename}.tif"

# # Ensure 'feature_max' is correctly defined (use a known value or calculate it)
# #feature_max = 100  # Example value, adjust based on actual maximum height from your data

# def load_and_preprocess_image(path):
#     with rasterio.open(path) as src:
#         image = src.read(1)
#     print("Max value in image before normalization:", np.max(image))  # Debug print
#     image = np.array(image).astype('float32') / feature_max
#     image = np.expand_dims(image, axis=-1)
#     image = np.expand_dims(image, axis=0)
#     return image

# input_image = load_and_preprocess_image(input_image_path)
# predicted_image = model.predict(input_image)
# predicted_image = np.squeeze(predicted_image)

# # Debug print to check if all outputs are the same
# print("Unique values in predicted output:", np.unique(predicted_image))

# predicted_image *= 100

# output_image_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\testexample\\predictions\\predicted_tile_{tilename}.tif"
# with rasterio.open(input_image_path) as src:
#     profile = src.profile

# with rasterio.open(output_image_path, 'w', **profile) as dst:
#     dst.write(predicted_image.astype(rasterio.uint8), 1)

# print("Done")


In [ ]:
import os
import re

# Define the directory path
directory_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\hag'

# Regular expression to extract the identifier part of the filename 'tile_{identifier}.tif'
pattern = re.compile(r'tile_(\d+-\d+)\.tif')

# List all files in the directory
files = os.listdir(directory_path)

# Use a set to avoid duplicate identifiers
identifiers = set()

# Extract identifiers from filenames
for file in files:
    match = pattern.search(file)
    if match:
        identifiers.add(match.group(1))

# Convert the set to a sorted list
identifier_list = sorted(list(identifiers))

#identifier_list


In [ ]:
import os
import rasterio
import numpy as np
import tensorflow as tf

#tilename = '0-0'
# input_hag_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\hag\\tile_{tilename}.tif"
# input_dem_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\dem\\tile_{tilename}.tif"
# input_roads_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\roads\\tile_{tilename}.tif"
# input_rivers_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\rivers\\tile_{tilename}.tif"

def load_and_preprocess_image(hag_path, dem_path, roads_path, rivers_path):
    with rasterio.open(hag_path) as src:
        hag_image = src.read(1)
    with rasterio.open(dem_path) as src:
        dem_image = src.read(1)
    with rasterio.open(roads_path) as src:
        roads_image = src.read(1)
    with rasterio.open(rivers_path) as src:
        rivers_image = src.read(1)

    # Normalize and stack the images
    hag_image = np.array(hag_image).astype('float32') / hag_max
    dem_image = np.array(dem_image).astype('float32') / dem_max
    roads_image = np.array(roads_image).astype('float32') / roads_max
    rivers_image = np.array(rivers_image).astype('float32') / rivers_max

    # Stack images along the last dimension
    combined_image = np.stack([hag_image, dem_image, roads_image, rivers_image], axis=-1)

    # Add batch dimension
    combined_image = np.expand_dims(combined_image, axis=0)
    return combined_image



for i in range(1000):
    tilename = identifier_list[i]
    print(tilename)
    input_hag_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\hag\\tile_{tilename}.tif"
    input_dem_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\dem\\tile_{tilename}.tif"
    input_roads_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\roads\\tile_{tilename}.tif"
    input_rivers_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\rivers\\tile_{tilename}.tif"


    input_image = load_and_preprocess_image(input_hag_path, input_dem_path, input_roads_path, input_rivers_path)
    predicted_image = model.predict(input_image)
    predicted_image = np.squeeze(predicted_image)
    
    # Debug print to check if all outputs are the same
    print("Unique values in predicted output:", np.unique(predicted_image))
    
    # Adjust the scaling factor based on how the labels were scaled during training
    predicted_image *= 100
    
    output_image_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\testexample\\predictions\\predicted_tile_{tilename}.tif"
    
    with rasterio.open(input_dem_path) as src: 
        profile = src.profile
    
    with rasterio.open(output_image_path, 'w', **profile) as dst:
        dst.write(predicted_image.astype(rasterio.uint8), 1)

print("Done")


In [ ]:
# import os
# import rasterio
# import numpy as np
# import tensorflow as tf

# tilename = '0-0'
# input_dem_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\dem\\tile_{tilename}.tif"
# input_hag_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\hag\\tile_{tilename}.tif"

# def load_and_preprocess_image(dem_path, hag_path):
#     with rasterio.open(dem_path) as src:
#         dem_image = src.read(1)
#     with rasterio.open(hag_path) as src:
#         hag_image = src.read(1)
    
#     print("Max value in DEM image before normalization:", np.max(dem_image))
#     print("Max value in HAG image before normalization:", np.max(hag_image))

#     # Normalize each image independently
#     dem_image = np.array(dem_image).astype('float32')
#     hag_image = np.array(hag_image).astype('float32')

#     if np.max(dem_image) > 0:  # Prevent division by zero
#         dem_image /= np.max(dem_image)
#     if np.max(hag_image) > 0:  # Prevent division by zero
#         hag_image /= np.max(hag_image)

#     # Stack images along the last dimension
#     combined_image = np.stack([dem_image, hag_image], axis=-1)

#     # Add batch dimension
#     combined_image = np.expand_dims(combined_image, axis=0)
#     return combined_image

# input_image = load_and_preprocess_image(input_dem_path, input_hag_path)
# predicted_image = model.predict(input_image)
# predicted_image = np.squeeze(predicted_image)

# # Debug print to check if all outputs are the same
# print("Unique values in predicted output:", np.unique(predicted_image))

# # Adjust the scaling factor based on how the labels were scaled during training
# predicted_image *= 100  # Adjust this factor according to your specific label scaling

# output_image_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\testexample\\predictions\\predicted_tile_{tilename}2.tif"
# with rasterio.open(input_dem_path) as src:  # You can use either DEM or HAG profile, assuming they're identical
#     profile = src.profile

# with rasterio.open(output_image_path, 'w', **profile) as dst:
#     dst.write(predicted_image.astype(rasterio.uint8), 1)

# print("Done")


In [ ]:
### WORKS WITH NO BOARDER!
import os
import glob
import subprocess

#tif_dir = r"D:\GlobalPCL\lidarHAG"
#output_tif = r"D:\GlobalPCL\lidarHAG\LIDAR_GEDI_CO.tif"
tif_dir = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\testexample\predictions"
output_tif  = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\testexample\merged\predMerged_CO_500.tif"


tifs = glob.glob(os.path.join(tif_dir, "*.tif"))
tifs = tifs[:300]

# Prepare the gdal_merge command for HAG
merge_command_hag = [
    "python",
    "C:\\Users\\smdur\\anaconda3\\envs\\globalpcl\\Scripts\\gdal_merge.py",
    "--config", "CHECK_DISK_FREE_SPACE", "FALSE",
    #"-ot", "Byte",
    "-o", output_tif,
    "-n", "-9999",
    "-a_nodata","-9999",
    
    
] + tifs

# Run the gdal_merge command for HAG and capture the output
process_hag = subprocess.run(merge_command_hag, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Check if the command for HAG was successful
if process_hag.returncode != 0:
    # An error occurred, print the error
    print("Error occurred while merging TIFF files HAG:")
    print(process_hag.stderr)
else:
    print("TIFF files merged successfully for HAG.")



In [ ]:
# import os
# import glob
# import subprocess

# tif_dir = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\testexample\predictions"
# output_tif  = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\testexample\merged\predMerged_CO.tif"

# # Get a list of TIFF files
# tifs = glob.glob(os.path.join(tif_dir, "*.tif"))

# # Write the list of TIFF files to a text file
# tif_list_file = os.path.join(tif_dir, "tif_list.txt")
# with open(tif_list_file, "w") as file:
#     file.write("\n".join(tifs))

# # Prepare the gdal_merge command with the text file
# merge_command_hag = [
#     "python",
#     "C:\\Users\\smdur\\anaconda3\\envs\\globalpcl\\Scripts\\gdal_merge.py",
#     "--config", "CHECK_DISK_FREE_SPACE", "FALSE",
#     #"-ot", "Byte",
#     "-o", output_tif,
#     "-n", "255",
#     "-a_nodata", "255",
#     "-of", "GTiff",  # Specify output format
#     "-l", f"@{tif_list_file}",  # Use the text file as input
# ]

# # Run the gdal_merge command and capture the output
# process_hag = subprocess.run(merge_command_hag, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# # Check if the command was successful
# if process_hag.returncode != 0:
#     # An error occurred, print the error
#     print("Error occurred while merging TIFF files:")
#     print(process_hag.stderr)
# else:
#     print("TIFF files merged successfully.")
